# K Clustering on graphs. 
Si consideri un grafo G=(V,E) non completo. 
Ogni partizione di V in k sottoinsiemi V 1 ,..,V k , indice k sottografi G(V 1 ),..G(V k ). Si determini la partizione tale per cui il numero di archi i cui estremi appartengono a sottografi diversi (i.e., la dimensione del multi-cut) è minimo.

In [173]:

# librerie
import networkx as nx
import matplotlib.pyplot as plt
from pulp import *
import numpy as np
import community
import itertools
import pprint

from networkx.algorithms import approximation

In [2]:
import graph
from gurobipy import *

def multi_cut_native(g):
    """return optimal multi_cut by solve an IP"""
    vertices = g.vertices()
    edges = g.edges()
    weights = g.weights()

    m = Model("muticut")

    # Create variables
    cuts = {} 
    for e in edges:
        e = frozenset(e)
        cuts[e] = m.addVar(vtype = GRB.BINARY) # if we cut edge e

    # Integrate new variables
    m.update()

    # Set objective
    m.setObjective(sum(cuts[e]*weights[e] for e in edges), GRB.MINIMIZE) # minimize cut-cost

    # Add constraints:
    for s,t in g.sts():
        for P in g.find_all_paths(s,t):
            m.addConstr( sum([ cuts[frozenset([P[j],P[j+1]])] for j in range(len(P) - 1)] ) >= 1 )

    # optimize it
    m.optimize()

    # # display results
    # def printSolution():
    #     print 'Problem size: |V| = ', n, ', |E|, = ', len(edges_weights) # size of our problem
    #     if m.status == GRB.Status.OPTIMAL:
    #         print('\nCost: %g' % m.objVal)
    #         #buyx = m.getAttr('x', buy)
    #         #nutritionx = m.getAttr('x', nutrition)
    #         print('\nCut:')
    #         for e in edges:
    #             print(e, cuts[e].x)
    #     else:
    #         print('No solution')

    # printSolution()
    x = {}
    for e in edges:
        x[e] = cuts[e].x
    return x